<a href="https://www.kaggle.com/code/shahrish99/resnet18-cifar100?scriptVersionId=256937641" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

# Set up data augmentation and preprocessing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load CIFAR-100 dataset
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

# Load ResNet-18 pre-trained on ImageNet
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer for CIFAR-100 (100 classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate Top-k accuracy
def calculate_topk_accuracy(output, target, topk=(1, 5)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Training and evaluation loop
num_epochs = 10

for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    top1_correct = 0
    top5_correct = 0
    total_samples = 0

    with tqdm(trainloader, desc=f"Epoch {epoch + 1}/{num_epochs} [Train]") as pbar:
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            # Resize images to match pre-trained model input dimensions
            inputs = nn.functional.interpolate(inputs, size=(224, 224))

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            total_samples += labels.size(0)

            # Calculate Top-1 and Top-5 accuracy
            top1, top5 = calculate_topk_accuracy(outputs, labels, topk=(1, 5))
            top1_correct += top1.item() * labels.size(0) / 100.0
            top5_correct += top5.item() * labels.size(0) / 100.0

            pbar.set_postfix({
                'Loss': running_loss / total_samples,
                'Top-1 Accuracy': 100.0 * top1_correct / total_samples,
                'Top-5 Accuracy': 100.0 * top5_correct / total_samples
            })

    # Evaluation Phase
    model.eval()
    test_loss = 0.0
    top1_correct = 0
    top5_correct = 0
    total_samples = 0

    with tqdm(testloader, desc=f"Epoch {epoch + 1}/{num_epochs} [Test]") as pbar:
        with torch.no_grad():
            for inputs, labels in pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = nn.functional.interpolate(inputs, size=(224, 224))

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                test_loss += loss.item()
                total_samples += labels.size(0)

                # Calculate Top-1 and Top-5 accuracy
                top1, top5 = calculate_topk_accuracy(outputs, labels, topk=(1, 5))
                top1_correct += top1.item() * labels.size(0) / 100.0
                top5_correct += top5.item() * labels.size(0) / 100.0

                pbar.set_postfix({
                    'Loss': test_loss / total_samples,
                    'Top-1 Accuracy': 100.0 * top1_correct / total_samples,
                    'Top-5 Accuracy': 100.0 * top5_correct / total_samples
                })

# Save the model
torch.save(model.state_dict(), 'resnet18_cifar100_tqdm.pth')

Files already downloaded and verified
Files already downloaded and verified


Epoch 10/10 [Test]: 100%|██████████| 79/79 [00:09<00:00,  8.15it/s, Loss=0.00881, Top-1 Accuracy=73.1, Top-5 Accuracy=93.4]


In [4]:
!pip install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-b

In [14]:
import torchattacks

# Set the model to evaluation mode
model.eval()

# Define attack methods
pgd_attack = torchattacks.PGD(model, eps=0.3, alpha=2/255, steps=40)
cw_attack = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
fgsm_attack = torchattacks.FGSM(model, eps=0.3)

# Helper function to evaluate the model on adversarial examples
def evaluate_adversarial_attack(attack, attack_name, dataloader):
    total_samples = 0
    correct_top1 = 0
    correct_top5 = 0

    print(f"\nEvaluating {attack_name} Attack...")
    for images, labels in tqdm(dataloader, desc=f"{attack_name} Attack"):
        images, labels = images.cuda(), labels.cuda()

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Evaluate on the adversarial examples
        outputs = model(adv_images)
        _, predicted = outputs.topk(5, 1, True, True)

        total_samples += labels.size(0)
        correct_top1 += (predicted[:, 0] == labels).sum().item()
        correct_top5 += sum([1 if labels[i] in predicted[i] else 0 for i in range(labels.size(0))])

    top1_acc = 100 * correct_top1 / total_samples
    top5_acc = 100 * correct_top5 / total_samples
    print(f"{attack_name} Top-1 Accuracy: {top1_acc:.2f}%")
    print(f"{attack_name} Top-5 Accuracy: {top5_acc:.2f}%")
    return top1_acc, top5_acc

pgd_top1, pgd_top5 = evaluate_adversarial_attack(pgd_attack, "PGD", testloader)
cw_top1, cw_top5 = evaluate_adversarial_attack(cw_attack, "CW", testloader)
fgsm_top1, fgsm_top5 = evaluate_adversarial_attack(fgsm_attack, "FGSM", testloader)


Evaluating PGD Attack...


PGD Attack: 100%|██████████| 79/79 [01:21<00:00,  1.03s/it]


PGD Top-1 Accuracy: 0.00%
PGD Top-5 Accuracy: 0.02%

Evaluating CW Attack...


CW Attack: 100%|██████████| 79/79 [00:55<00:00,  1.43it/s]


CW Top-1 Accuracy: 1.34%
CW Top-5 Accuracy: 10.13%

Evaluating FGSM Attack...


FGSM Attack: 100%|██████████| 79/79 [00:03<00:00, 24.91it/s]

FGSM Top-1 Accuracy: 0.22%
FGSM Top-5 Accuracy: 2.11%
